<div style='background-color:#f7f7f7; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px'>
    <center>
        <div style='  display: block;
  font-size: 2em;
  font-weight: bold;  display: block;
  font-size: 2em;
  font-weight: bold;'>The Bayesian Blocks Algorithm
        </div>
    <center>
        <br>
    <i>Gabriele Bertinelli - Matteo Lovato</i></center>
    <i>2103359 - 2104269</i>
    <center>

### Abstract

The Bayesian Blocks algorithm can be used to improve the binning of histograms by finding the optimal segmentation of the data. The visual improvement can be dramatic, but, more importantly, this algorithm produces histograms that accurately represent the underlying distribution while being robust to statistical fluctuations.
In this project, the Bayesian Blocks algorithm is implemented in R and its performances are tested with different sets of data.

<!---### Index <a class="anchor" id="0"></a>

1. [Introduction](#1) <br> 
    1.1. [Representation of data: histograms](#1.1) <br> 
    1.2. [Introduction to Bayesian Blocks method](#1.2) <br> 
2. [Main assumptions of Bayesian Blocks method](#2) <br>
    2.1. [Fitness of Blocks](#2.1) <br> 
    2.3. [Prior Distribution](#2.2) <br> 
3. [The Algorithm](#3) <br> 
    3.1. [Description](#3.1) <br> 
    3.2. [Implementation](#3.2) <br> 
    3.3. [Computational Time](#3.3) <br> 
4. [Applications](#4) <br> 
    4.1. [Mix of Normal Distributed Variables](#4.1) <br> 
    4.2. [Uniform Background and Gaussian Signals](#4.2) <br> 
    4.3. [ECG Data](#4.3) <br> 
    4.4. [HPGe Eu152 energy spectrum](#4.4) <br>
5. [KDE vs Bayesian Blocks](#5)<br>
6. [Conclusions](#6) <br> 

----------------------- --->

## 1. Introduction <a class="anchor" id="1"></a>

### 1.1. Representation of data: histograms<a class="anchor" id="1.1"></a>

The **data analysis goal** is to identify and characterize statistically significant variations and features in observed data. In particular, histograms are used to represent distributions of data. 
    
Most of the time, a subjectively natural range and bin width is chosen, motivated mainly by obtaining a nice looking plot.
Objective methods have been proposed that determine binning according to some optimization procedure such as   Scott’s Rule optimal for random samples of normally distributed data.

* Some methods takes the **structure of the distribution** into account but uses bins of **fixed width**. 

* Others requires that **each bin have similar numbers of entries**, and thus the bin widths may vary, but the **location of the bin edges** is still chosen **arbitrarily**.


<!---<div class="alert alert-block alert-success"> --->
<u>The <b>Bayesian Blocks algorithm</b>, in contrast, allows the bin widths to vary and determines the bin edges based on the structure of the distribution. 
</u>

### 1.2. Introduction to Bayesian Blocks method<a class="anchor" id="1.2"></a>

The Bayesian Block algorithm was originally developed by Jeffrey D. Scargle for applications in astronomy to addresses the problem of detecting and characterizing local variability in time series. Although it was developed for time series data, the algorithm is applicable to other forms of sequential data, or any other other independent variable.
In particular, it can be used to improve the representations of histograms into ones in which the bins are not fixed and are free to be unequal in size as determined by the data.

<!---<div class="alert alert-block alert-info">--->
The Bayesian Blocks algorithm is a <b>non-parametric</b> modeling technique for determining the optimal segmentation of a given set of univariate random variables into blocks, with each block containing consecutive data elements satisfying some well defined criterion. It operate in a <b>bayesian framework</b>.
</div>

The <b>goal</b> is to separate statistically significant features from the ever-present random observational errors and thus to discover local structures in background data exploiting the full information brought by the data themself.


These concepts and methods can be applied in general higher dimensional contexts. Here, however, we concentrate on one-dimensional data.

## 2. Main assumptions of Bayesian Blocks method<a class="anchor" id="2"></a>

<b>Main idea</b>: segmentation of the data interval into variable-sized blocks, each containing consecutive data satisfying some well defined criteria. The optimal segmentation is the one that maximizes some quantification of this criterion.

Each block (or bin, in the context of histograms) is consistent with a probability distribution with compact support; the entire dataset is represented by this collection of finite probability distribution functions.
In the algorithm implementation each block is modeled with a uniform distribution which defines the so called `Piecewise Constant Model`.
By optimizing a goodness-of-fit statistic which depends only on the input data and a regularization parameter that envelop our prior knowledge, the number of blocks and the edges of the blocks, called `change points`, can be computed. Depending on the kind of data the fitness function changes.
The segmentation of the whole observation interval is described by the following parameters:

* $N_{cp}$, the number of change-points.

* $t_k^{cp}$, the change-point starting block $k$.

* $X_k$, the signal amplitude in block $k$.

for $k=1,2,\dots,N_{blocks}$, where the number of blocks is $N_{blocks}=N_{cp}+1$.

|Data type| Description|
|:---|:---|
|Events (TTE)| Time at which the measurements occured|
|Binned Events| Counts of events in time bins|
|Point Measurements |Measurements of a quasi-continuous observable at a sequence of points in time|

The set of blocks is gapless and non-overlapping, where the first block edge is defined by the first data point, and the last block edge is defined by the last data point. A block can contain between 1 and N data points, where the sum of the contents of all the blocks must equal N . The algorithm relies on the additivity of the fitness function, and thus the fitness of a given set of blocks is equal to the sum of the fitnesses of the individual blocks. The total fitness, $F_{total}$ for a given dataset is:

$$F_{total} = \sum_{i=1}^{K} f(B_i)$$

$f(B_i)$ is the fitness of block $i$. The latter can be any convenient measure of how well a constant signal represents the data within the block.

### 2.1. Fitness of Blocks <a class="anchor" id="2.1"></a>
As stated before, depending of the kind of data the fitness function assumes different forms.

#### **Events**

The fitness function can be obtained starting with the unbinned likelihood known as the Cash statistic <a name="cite_note-6"><sup>[6]</sup></a>:

$$\large \log L(\theta) = \sum_{n} \log M (t_n, \theta) - \int M (t, \theta)dt$$

where M(t, θ) is a model of the time dependence of a signal.
The integral is over the observation interval and is the expected number of events under the model.
Since each $k$-th block is constant, and so has a single parameter $M(t, \theta) = \theta$, the resulting log-likelihood is:

$$\large \log L^{(k)}(\theta) = N^{(k)}\log \theta - \theta T^{(k)}$$

where $N^{(k)}$ is the number of events in the $k$-th block and $T^{(k)}$ is the length of the block.</br>
When $\theta =N^{(k)}/T^{(k)} $ the fitness is maximum, yielding:

$$\large \log L^{(k)}_{\max}(\theta) +  N^{(k)} = N^{(k)}(\log N^{(k)} - \log T^{(k)})$$
 
Since the sum of $N^{(k)}$ is constant over $k$, the term on the left hand side is irrelevant since is model indipendent. Moreover, note that this fitness function is scale invariant $(t \to \alpha t)$.

<a name="cite_note-6"></a>6. [^](#cite_note-6) W., Parameter-Estimation in Astronomy
through Application of the Likelihood Ratio, Astrophysical
Journal.<br>

#### **Binned Event Data**

The expected count in a bin is the product $\lambda \cdot e \cdot W$ of the true event rate $\lambda$ at the detector, a dimensionless exposure factor $e$, and
the width of the bin $W$ . Therefore the likelihood for bin n is given
by the Poisson distribution:

$$\large L_n = \frac{(\lambda e_n W_n)^{N_n}e^{- \lambda e_n W_n}}{N_n !}$$

where $N_n$ is the number of events in bin $n$, $\lambda$ is the actual event
rate in counts per unit time, $e_n$ is the exposure averaged over the
bin, and $W_n$ is the bin width in time units. Defining bin efficiency
as $w_n = e_n W_n$, the likelihood for block $k$ is the product of the
likelihoods of all its bins:


$$\large L^{(k)} = \prod_{n=1}^{M^{(k)}} L_n = \lambda^{N^{(k)}} e^{-\lambda w^{(k)}}$$

where $M^{(k)}$ is the number of bins contained in block $k$,

$$\large w^{(k)} = \sum_{n=1}^{M^{(k)}} w_n$$

is the sum of the bin efficiencies in the block, and

$$\large N^{(k)} = \sum_{n=1}^{M^{(k)}} N_n$$

is the total event count in the block.</br>
Notice that the factor $(e_n W_n)^{N_n}/N_n !$ has been discarted since its product over all the bins in all the blocks is constant.
In the end the equation becomes:

$$\large \log L^{(k)}= N^{(k)}\log \lambda - \lambda w^{(k)}$$

#### **Point Measurements**

A common experimental scenario is to measure a signal $s(t)$ at a sequence of times $t_n, n = 1, 2, . . . , N$, known as sampling, in order to characterize its time dependence.
Inevitable corruption due to observational errors is frequently countered by smoothing the data and/or fitting a model.
As with the other data modes Bayesian Blocks is a different approach to this issue, making use of knowledge of the observational error distribution and avoiding the information loss entailed by smoothing.
We assume tha the measurements at these times are independent of each other, which is to say the errors of observation are statistically independent. </br>
Generally these errors are stochastic and addittive, so the measured time series can be modeled as follow:

$$\large x_n \equiv x(t_n) = s(t_n) + z_n$$

The error $z_n$ at time $t_n$ is known only by its distribution. Let us consider $z_n \sim \mathcal{N}(0, \sigma_n)$. If we model the signal as a constant $s = \lambda$, the likelihoood of the $n$-th measurement is:

$$ \large L_n = \frac{1}{\sigma_n \sqrt{2 \pi}} e^{- \frac{1}{2} \left( \frac{x_n - \lambda}{\sigma_n}\right)^2}$$

So, since we assume the indipendence of the measurements, the block $k$ lielihood is:

$$\large L^{(k)} = \prod_n L_n = \frac{(2 \pi)^{- \frac{N_k}{2}}}{\prod_m \sigma_m} e^{-\frac{1}{2} \sum_n \left( \frac{x_n - \lambda}{\sigma_n}\right)^2}$$

Notice that the quantity multiplying the exponential in the above equation is irrelevant because it contributes an overall constant factor to the total likelihood.</br>
Following the computation in the original paper, the maximum value for the log-likelihood is given by:

$$\large \log L^{(k)}_{\max} = b_{k}^{2}/4 a_k$$

where $a_k$ and $b_k$ are defined as follows:

$$ a_k = \frac{1}{2} \sum_n \frac{1}{\sigma^2_n}$$
$$ b_k = \frac{1}{2} \sum_n \frac{x_n}{\sigma^2_n}$$

### 2.2. Prior Distribution<a class="anchor" id="2.2"></a>

<!---<div class="alert alert-block alert-info"> --->
The fitness described above must be modified by a penalty term for the number of blocks. 
We influence the number of blocks by defining a prior distribution for the number of blocks. Adjusting a parameter controlling the steepness of this prior establishes relative probabilities of smaller or larger numbers of blocks.
</div>

In the usual fashion for Bayesian model selection, in cases with high signal-to-noise, $N_{blocks}$ is determined by the structure of the signal; with lower signal-to-noise the prior becomes more and more important. In short, we are regulating not smoothness but complexity.

#### 2.2.1. Geometric Prior
In most settings, it is much more likely a prior for which $N_{blocks} \ll N$, than $N_{blocks} \sim N$.
For this reason, it is desirable to impose a prior that assigns smaller probability to a large number of blocks, and we adopt a **geometric prior** with parameter $\gamma$:

$$P(N_{blocks}) = P_0 \gamma^{N_{blocks}} $$

where $P_0$ represent the normalization constant, and it is easily obtained as follows:

$$P(N_{blocks}) = \frac{1-\gamma}{1-\gamma^{N+1}} \gamma^{N_{blocks}}$$
and the expeceted number of blocks is:

$$ <N_{blocks}> = P_0 \sum_{N_{blocks}}^{N} N_{blocks} \gamma^{N_{blocks}} = \frac{N\gamma^{N+1}+1}{\gamma^{N+1}-1} + \frac{1}{1-\gamma}  $$
 
It is possible to note that the estimated number of blocks is a discontiunuos monotonic function of $\gamma$, and for values greater than 1 it leads almost certanily to assigning each datum to a separate block. 

The contribution of the prior to the fitness of each block can be implemented simply by adding the constant $log(\gamma)$ (refered to as `ncp_prior`) to the fitness of each block as follow:

$$F_{total} = \sum_{i=1}^{K} f(B_i) = \sum_{i=1}^{K} log L^{i} + log(\gamma) $$

#### 2.2.2. Fixing the Parameter $\gamma$

The choice of $\gamma$ influences the visual representation, adjusting the amount of structure in the blocks, so it is worth to brifly discuss its effects and how to properly choose it. In many application the results are rather insensitive to the value of $\gamma$, as long as the signal to noise ratio is moderatly large, but for extreme value, i.e. ($\gamma \geq$ 1), it can lead to bad results, like too few or too many blocks. The tread off that one must consider when choising the prior is between a conservative choice for the number of blocks, resilient to flactuation but possibly missing real changes, and a liberal choice, which is able to capture more changes and also is more prone to false detections. As suggested by Scargle et al.<a name="cite_note-5"><sup>[5]</sup></a>, an objective method can be implemented as a function of the number of data points $N$, based on the fact that the `ncp_prior` can be seen as a proxy for the false alarm rate $p_0$, which is defined as the frequency with which the algorithm correctlty rejects the presence of a change point. 

#### 2.2.3. Rigorous Calibrated Prior for Event Data

A prior calibration following the rigorous approach for **Event Data** (i.e. for which one data cell is associated to each event) yields:

$$
\log P(N,p_0) = 4 - \log(73.53 p_0 N^{-0.478}) 
$$

## 3. The Algorithm <a class="anchor" id="3"></a>

### 3.1. Description <a class="anchor" id="3.1"></a>

Given an ordered set of $N$ data points, the algorithm determines the optimal set of $K + 1$ change-points (and therefore $K$ blocks) which partition the observation intervall,  by iterating through the data points, and caching the current maximum fitness values and
corresponding indices.
At most there can be $N$ total blocks, one for each point.
It can be proven that given $N$ data points the total number of possible partitions of the observation interval is $2^N$. Following the orignal work, it can be shown that removing the last block of an optimal partition leaves an optimal partition. This allows to iteratively consider an increasing number of data points and at each step compute the optimum partition for them, that will be used for the next iteration, thus reducing the space of the possible partition to check drammatically. The computational time of the algorithm is $O(N^2)$.

<center>
<div>
    <img src="../images/algo.png" width="900" />
</div>
<em><strong>Figure 1</strong>: Representation of the iteration over the possible optimal partition of the observation intervall. The red dots represents the events, while the black ticks over the timeline indicate the possible change points.</em>
</center>

### 3.2. Implementation <a class="anchor" id="3.2"></a>

For our implementation we have taken inspiration from both the ```MATLAB``` code reported in Scargle et al.<a name="cite_note-5"><sup>[5]</sup></a> and from the ```astropy``` source code for the Bayesian Blocks algorithm. 

We omit our source code for brevity. You can find the script in the file ```func_bb.R```.

### 3.3. Computational Time <a class="anchor" id="3.3"></a>

The computational time required by the Bayesian Blocks algorithm scales as $O(N^2)$.\
An increasing number of uniformly distributed events have been generated over the intervall $[0,100)$ and for each one of them have been computed the edges of the bayesian blocks and the time it took to finish the calculations. In order to build some statistics for every data size the algorithm has been run 10 times. The data produced have been fitted with a parabola with its vertex on the origin.

<center>
<div>
<img src="../images/benchmark_plot.png" width="500" />
</div>
<em><strong>Figure 2</strong>: Algorithm computational time test.</em>
</center>

## 4. Applications<a class="anchor" id="4"></a>
Now, we test the Bayesian Blocks algorithm with different sets of data in order to evaluate its performance.

### 4.1. Mix of Normal Distributed Variables<a class="anchor" id="4.1"></a>


We sampled a set of normal distributed random data, which dimension is $N = 13000$.\
We plotted together: 
- the histogram of data with equal bin size and with a number of edges chosen to be $\sqrt{N}$
- the edges calculated with the BB algorithm, with the two different implemented priors: $\gamma$ and $p_0$.\

We can see that the adaptive-width bins lead to a very clean representation of the important features in the data. Moreover, if we tune appropriately the parameter's prior, we cannot note any difference between them. In practice, as the length of the data becomes higher, the used prior influence less the representation of the data. 

<center>
<div>
<img src="../images/test_mix_normal.png" width="1000" />
</div>
<em><strong>Figure 3</strong>: Comparison between an arbitrary number of edges and the BB algorithm. Left: we choose a $\gamma=0.001$ prior. Right: we choose the default $p_0=0.05$ prior. The number of edges given by the priors is comparable.</em>
</center>

### 4.2. Uniform Background and Gaussian Signals<a class="anchor" id="4.2"></a>

Now, we test the algorithm on a dataset made by a uniform noise background and three gaussian signals. In particular, we want to see if the algorithm is capable to recognize both tight and wide peaks.

As you can see in the Figure belowe, the algorithm is able to recognize all the important information of the dataset. It recognize very well tight peak. It also recognize the gaussian wider peak by distinguishing well the uniform background.

<center>
<div>
<img src="../images/test_noise.png" width="1000" />
</div>
<em><strong>Figure 4</strong>: Histogram of uniform background and gaussian signals. Upper: we choose a $\gamma=0.001$ prior. Lower: we choose the default $p_0=0.05$ prior. The number of edges given by the priors is comparable. We can exploit the simplicity of this algorithm: it returns with few number of edges the most important features of the dataset.</em>
</center>

### 4.3. ECG Data<a class="anchor" id="4.3"></a>

Using the Bayesian Blocks on point measurements type of data we thought to test if was possible to denoise an ECG track without loosing any meaningful information. The data used comes from Kaggle <a name="cite_note-10"><sup>[10]</sup></a>. As can be qualitatively seen by setting `ncp_prior` = 100, thus forcing the representation to use less blocks, the result representation computed by the algorithm perfetcly reproduce the shape of the signal, keeping its key features, while ignoring its small fluctuations and flattening the region with no informations.


<a name="cite_note-10"></a>10. [^](#cite_note-10) Data from: https://www.kaggle.com/datasets/stetelepta/sample-ecg-data?resource=download

<center>
    <div>
<img src="../images/ecg_plot.png" width="1000" />
        </div>
    <em><strong>Figure 5</strong>: On top is shown the ECG signal while on the botton is plotted the histogramm over the track.
    </em>
</center>

### 4.4. HPGe Am241, Co60 and Cs137 energy spectrum <a class="anchor" id="4.4"></a>

Let us test the Bayesian Blocks algorithm with an energy spectrum dataset collected with an HPGe detector. The source is a combination of Am241, Co60 and Cs137. It was sealed up and only photons could escape from the source to be detected. Indeed, the alpha particles were absorbed by the sealing. 

The Bayesian Blocks algorithm highlitghts the main features of the energy spectrum by recognizing both the main spikes of the spectrum and the continuum. These peaks correspond to the energy levels of the radioactive isotopes (Am241, Co60, and Cs137).
We note also that varying the parameter of the prior ($\gamma$, $p_0$ or ```ncp_prior```) does not change the result of the algorithm. It is due to the high number of data in the histogram. As said, for high statistics the prior becomes less important in the spirit of Bayesian analysis.

In the figure below we isolated a region of the spectrum for better visualization of the Bayesian Blocks. It must be noted that, even trying to find the best parameter, the algorithm struggled identifying block in the tails of the spectrum. This could be due to a sub-optimal implementation of the algorithm or due to the dataset itself.


<center>
<div>
    <img src="../images/spec1_plot.png" width="1000" />
</div>
    <em><strong>Figure 6</strong>: Upper: plot of the energy spectrum of Am241, Co60 and Cs137 collected with an HPGe detector. Lower:We overimposed to the spectrum (blue line) the Bayesian Block representation (red line). We adopted a <tt>'PointMeasures' data_type</tt> and <tt>ncp_prior=0</tt>. The x-axis is in arbitrary units. The scale on y-axis is logarithmic (base 10) and we selected the counts from 1500 to 5000 for a better visualization of the result.
    </em>
</center>

### 4.5. HPGe Eu152 energy spectrum<a class="anchor" id="4.5"></a>

Now, let us test the Bayesian Blocks algorithm with an energy spectrum dataset collected with a HPGe detector with a Eu152 source.  
This time we have less statistics with respect to the case above.

We note that the algorithm recognize the spikes (with a little more difficulty on the right tail). However the representation is not optimal, because the Bayesian Blocks do not retain the real width of the peaks. It could be due to the large number of spikes present in this plot. Indeed, they are a lot and they are also not well distinguishable from the background. We recall that a lower signal-to-noise ratio can influence a lot the performance of the Bayesian Blocks algorithm.

<center>
<div>
    <img src="../images/spec2_plot.png" width="1000" />
</div>
    <em><strong>Figure 7</strong>: Plot of the energy spectrum of Eu152 collected with an HPGe detector (blue line). We binned the counts to create an histogram. We overimposed the Bayesian Blocks representation (red line).  We adopted a <tt>'Events' data_type</tt> and <tt>ncp_prior=150</tt>. The x-axis is in arbitrary units. The scale on y-axis is logarithmic (base 10) for a better visualization of the result.
    </em>
</center>

## 5. KDE vs Bayesian Blocks <a class="anchor" id="5"></a>

#### **Kernel Density Estimation**
Kernel density estimation (KDE) is a technique for estimation of probability density function that enable the user to have a different view on data that traditionally would be histogrammed.
Unlike the histogram, the kernel technique produces smooth estimate of the pdf, uses all sample points' locations and more convincingly suggest multimodality<a name="cite_note-7"><sup>[7]</sup></a>.


<a name="cite_note-7"></a>7. [^](#cite_note-7)Węglarczyk, Stanisław. "Kernel density estimation and its application." ITM web of conferences. Vol. 23. EDP Sciences, 2018.<br>

#### **Comparison**
Here we report the comparison between the visualizations of some synthetic data obtained with the Bayesian Block algorithm and the Kernel Density Estimation (KDE) method.

As can be seen from the figure belowe, even though the KDE representation seems to yield the correct location of the distributions' peaks, it drammatically fails to represent correctly the variance of the most sharp one, moreover the right tail of the distribution falls while the true pdf presents a uniform behaviour.
Meanwhile the Bayesian Block clearly represents the most sharp peak, both its position and variance, while indicating the presence on the wider one, despite beeing less clear with respect to the KDE visualization. The algorithm prefectly shows the uniform distributed reagions.

<center>
<div>
    <img src="../images/kde_vs_bb.png" width="1000" />
</div>
    <em><strong>Figure 8</strong>: The background is been sampled from a uniform distribution ranging from 0 to 100 for a total of 5000 points, while the two signals consists in 700 points each sampled from two gaussian distribution: the first with $\mu = 10$ and $\sigma=4$, while the second one with $\mu = 40$ and $\sigma=1$
    </em>
</center>

## 6. Conclusions<a class="anchor" id="6"></a>

In conclusion, the Bayesian Blocks representation provides an objective way to enlighten the key features of a data set by imposing few preconditions as possible.

An advantage of Bayesian Blocks is that, most of the times, the adaptive-width bins lead to a very clean representation of the important features in the data. Moreover, the algorithm can center the peak of a signal in a bin, so that it will be as clear as possible compared to the background. 

However, Bayesian Blocks are better if you are interested in peaks than tails, as tails have few entries and therefore do not change much and have few change points.
For the same reason, Bayesian Blocks work better the more data you have, as the changes can be too small to make change points in small data sets. Thus, we need large statistics to correct recognize the important features of a histogram.

Further development on the algorithm will be to implement it in such a way that will enable to study multivariate data as well further investigate for which application this method represent a better visualization with respect to the KDE and vice versa.